In [3]:

import jax
import jax.numpy as jnp
import flax
import tokenizer

# from tokenizer import tokenizeLine
# tokenizer.tokenize()

vocab = tokenizer.makeVocabUCI()
games = open('data/ELO_2000_UCI.txt', 'r').read()
games = games.splitlines()

tokenizedGames = []

for g in games:
    arr = []
    for e in g.split(' '):
        arr.append(vocab[e])
    tokenizedGames.append(arr)

In [ ]:
import jax
import jax.numpy as jnp
import flax.linen as nn

# nn.
jax.nn.conv2d()
# nn.co